In [1]:
# We compare pairwise similarity performance on `spectra_file`
# We can use pickled version for much faster loading times
spectra_file = 'GNPS-random-10k.mgf'

# We take a random sample of spectra from said file

# Minimum size:
chunk_sizes_min = 32

# Maximum size
chunk_sizes_max = 20_000

# how many points to evaluate (in logspace) between min and max
num_evals = 15

! nvidia-smi -L
! echo Number of CPU cores $(nproc)

GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-22a456d4-1683-b2f7-77b8-3b3ae6eabb63)
Number of CPU cores 192


In [2]:
from cudams.utils import Timer
from tqdm import tqdm
import numpy as np
import sys
import pandas as pd
import pickle
from cudams.utils import download
from joblib import Parallel, delayed
from cudams.similarity import CudaCosineGreedy, CudaModifiedCosine
from matchms.importing import load_from_mgf
from matchms.filtering import require_precursor_mz
from cudams.utils import Timer

np.random.seed(42)

raw_spectra = list(load_from_mgf(download(spectra_file)))

In [3]:
spectra = []
for s in raw_spectra:
    s = require_precursor_mz(s)
    if s is not None:
        spectra.append(s)

In [4]:
len(spectra)

9997

In [5]:
chunk_sizes = np.round(np.logspace(
    np.log2(chunk_sizes_min), 
    np.log2(chunk_sizes_max), 
    num=num_evals, 
    base=2, 
    endpoint=True)
).astype(int)

def loop(chunk_size, kernel):
    references = np.random.choice(spectra, size=chunk_size)
    queries = np.random.choice(spectra, size=chunk_size)

    # Allow warm-up
    kernel.matrix(references[:4], queries[:4])

    with Timer() as timer:
        kernel.matrix(references, queries)
    return kernel.__class__.__name__, \
        timer.duration,\
        len(references) * len(queries) # All samples

# We only have 1 GPU, so no parallel here
data = Parallel(1)(delayed(loop)(chunk_size, kernel) for chunk_size in tqdm(chunk_sizes) for kernel in [CudaCosineGreedy(), CudaModifiedCosine()])
data = pd.DataFrame(data, columns=['kernel','time','pairs'])
data['label'] = 'RTX4090'

100%|██████████| 15/15 [04:39<00:00, 18.64s/it]


In [6]:
data.to_json(sys.stdout)

{"kernel":{"0":"CudaCosineGreedy","1":"CudaModifiedCosine","2":"CudaCosineGreedy","3":"CudaModifiedCosine","4":"CudaCosineGreedy","5":"CudaModifiedCosine","6":"CudaCosineGreedy","7":"CudaModifiedCosine","8":"CudaCosineGreedy","9":"CudaModifiedCosine","10":"CudaCosineGreedy","11":"CudaModifiedCosine","12":"CudaCosineGreedy","13":"CudaModifiedCosine","14":"CudaCosineGreedy","15":"CudaModifiedCosine","16":"CudaCosineGreedy","17":"CudaModifiedCosine","18":"CudaCosineGreedy","19":"CudaModifiedCosine","20":"CudaCosineGreedy","21":"CudaModifiedCosine","22":"CudaCosineGreedy","23":"CudaModifiedCosine","24":"CudaCosineGreedy","25":"CudaModifiedCosine","26":"CudaCosineGreedy","27":"CudaModifiedCosine","28":"CudaCosineGreedy","29":"CudaModifiedCosine"},"time":{"0":0.116838787,"1":0.288407699,"2":0.11716113,"3":0.290607731,"4":0.123106576,"5":0.295652899,"6":0.158863593,"7":0.324127188,"8":0.14879448,"9":0.349230335,"10":0.159278728,"11":0.381819613,"12":0.179009665,"13":0.425479235,"14":0.2160349